In [ ]:
### We need to install these libraries

! pip install mysql-connector-python
! pip install pandas

In [ ]:
### We import these libraries

import mysql.connector as connection
import pandas as pd

In [ ]:
### We connect to the MySql and database
try:
    mydb = connection.connect(host = "localhost",user= "root", passwd= "/Tatung7358", use_pure= True)
    # check if the connection is established

    query = "USE automobile"

    cursor = mydb.cursor() # create a cursor to execute queries
    cursor.execute(query)
    print(cursor.fetchall())

except Exception as e:       #### This will close our database and return an error
    mydb.close()
    print(str(e))

In [ ]:
## we want to use the select from cars query to test if our database has properly connected
cursor.execute("SELECT * FROM Cars")

output = cursor.fetchall() 

for x in output:
    print(x)

In [ ]:
### We want to present the table in a panda dataframe
query = "SELECT * FROM Cars"

cars_data = pd.read_sql(query, mydb)
cars_data

In [ ]:
### exploring the cars table

cars_data.head

In [ ]:
# Business problem 1:

# inserting a new service booking into the service table from a car that has been serviced previously 

insert_query = "INSERT INTO services VALUES ('S2006-138', '2020-06-20', '10:00:00', 'Oil change and filter replacement', 10421, '2021-06-20', 'STR 9378')"
cursor.execute(insert_query)
##connection.commit()
print ("service booking data is inserted")



In [ ]:
### Business problem 1 result
## To view the newly added service booking in the dataframe
query = "SELECT * FROM services"

services_data = pd.read_sql(query, mydb)

services_data

In [ ]:
# Business Problem 2:
# retrieveing data from service booking
retrieve_query = """SELECT M.name, COUNT(SM.service_id) AS JobsOnDropOffDay
FROM mechanics M 
JOIN service_mechanics SM ON M.employee_id = SM.employee_id
JOIN services S ON SM.service_id = S.service_id
WHERE S.registration_no = 'SEZ 5629'
  AND S.drop_off_date = '2020-06-20'
  GROUP BY M.name
HAVING COUNT(SM.service_id) > 0;"""
cursor.execute(retrieve_query)
results = cursor.fetchall()
columns = cursor.column_names
#print("""service booking:""")
#print(columns)
#for result in results:
#    print(result)

business_prob2 = pd.read_sql(retrieve_query, mydb)
business_prob2

In [ ]:
# Business Problem 3:
# the total service time per mechanic within a period of 24hrs time
retrieve_query = """SELECT M.name,  SEC_TO_TIME(SUM(TIME_TO_SEC(SM.time_spent))) AS TotalTimeSpent
FROM mechanics M 
JOIN service_mechanics SM ON M.employee_id = SM.employee_id
JOIN services S ON SM.service_id = S.service_id
WHERE S.drop_off_date BETWEEN '2020-06-18' AND '2020-06-19'
GROUP BY M.name;"""
cursor.execute(retrieve_query)
results = cursor.fetchall()
columns = cursor.column_names
print("""The total service time per mechanic within a period of 24hrs time""")
print(columns)
for result in results:
    print(result)

business_prob3 = pd.read_sql(retrieve_query, mydb)
business_prob3

In [ ]:
## Business problem 4: 
## Listing the name and email address of customers who own a car where the date of next service falls between two given dates
retrieve_query = """SELECT C.name AS customername, C.email_address AS CustomerEmail, Cars.Make, Cars.registration_no, S.next_service_date
FROM customers C
JOIN cars ON C.customer_id = cars.customer_id
JOIN services S ON cars.registration_no = S.registration_no
WHERE S.next_service_Date BETWEEN '2021-06-18' AND '2021-06-23';"""
cursor.execute(retrieve_query)
results = cursor.fetchall()
columns = cursor.column_names
print("""Listing the name and email address of customers who own a car where the date of next service falls between two given dates""")
print(columns)
for result in results:
    print(result)

business_prob4 = pd.read_sql(retrieve_query, mydb)
business_prob4

In [ ]:
## - Business problem 5: Making a mechanic unavailable, then getting the services they may be involved with during those days of unavailability
##  First, I need to  alter the table by adding a column on the availabilty status

alter_mechanics = """ALTER TABLE Mechanics
ADD COLUMN status VARCHAR(50) DEFAULT 'Available';"""

#### lets see the changes on the mechanic table
select_query = "SELECT * FROM mechanics"
mechanics_data = pd.read_sql (select_query, mydb)

## then, i want to change the status of a mechanic to unavailable, i will use Alan Turing with the ID no E2648 
update_alan = """UPDATE mechanics
SET Status = 'Unavailable'
WHERE employee_id = 'E2648';"""
cursor.execute(update_alan)
print("Alan availability is changed")

mechanics_data

In [ ]:
## Business prob 5:
# getting a list of any services Alan may be involved in on the days he is unavailable

retrieve_query = """SELECT S.service_id, S.drop_off_date, S.drop_off_time, S.description, S.registration_no
FROM services S
JOIN service_mechanics SM ON S.service_id = SM.service_id
JOIN mechanics M ON SM.employee_id = M.employee_id
WHERE M.Status = 'Unavailable'
  AND S.drop_off_date BETWEEN '2020-06-18' AND '2020-06-23';"""

##cursor.execute(retrieve_query)
##results = cursor.fetchall()
##columns = cursor.column_names
##print("""getting a list of any services Alan may be involved in on the days he is unavailable""")
##print(columns)
##for result in results:
##    print(result)

business_prob5 = pd.read_sql(retrieve_query, mydb)
business_prob5

In [ ]:
mydb.close()